# Stream CockroachDB CDC to Databricks (Azure)

This notebook demonstrates how to stream CockroachDB changefeeds to Databricks using Azure Blob Storage.

## Prerequisites

- CockroachDB cluster (Cloud or self-hosted)
- Azure Storage Account with hierarchical namespace enabled
- Databricks workspace with Unity Catalog
- Unity Catalog External Location configured for your storage account

**Note:** This notebook uses the **YCSB (Yahoo! Cloud Serving Benchmark)** schema as the default table structure, with `ycsb_key` as the primary key and `field0-9` columns. The default schema name is `public`.

## CDC Mode Selection

This notebook supports **4 CDC ingestion modes** by combining two independent settings:

### 1. CDC Processing Mode (`cdc_mode`)
How CDC events are processed in the target table:

- **`append_only`**: Store all CDC events as rows (audit log)
  - **Behavior**: All events (INSERT/UPDATE/DELETE) are appended as new rows
  - **Use case**: History tracking, time-series analysis, audit logs
  - **Storage**: Higher (keeps all historical events)

- **`update_delete`**: Apply MERGE logic (current state replication)
  - **Behavior**: DELETE removes rows, UPDATE modifies rows in-place
  - **Use case**: Current state synchronization, production replication
  - **Storage**: Lower (only latest state per key)

### 2. Column Family Mode (`column_family_mode`)
Table structure and changefeed configuration:

- **`single_cf`**: Standard table (1 column family, default)
  - **Changefeed**: `split_column_families=false`
  - **Files**: 1 Parquet file per CDC event
  - **Use case**: Most tables, simpler configuration, better performance

- **`multi_cf`**: Multiple column families (for wide tables)
  - **Changefeed**: `split_column_families=true`
  - **Files**: Multiple Parquet files per CDC event (fragments need merging)
  - **Use case**: Wide tables (50+ columns), selective column access patterns

### Function Selection Matrix

The notebook automatically selects the appropriate ingestion function based on your configuration:

| CDC Mode | Column Family Mode | Function Called |
|----------|-------------------|-----------------|
| `append_only` | `single_cf` | `ingest_cdc_append_only_single_family()` |
| `append_only` | `multi_cf` | `ingest_cdc_append_only_multi_family()` |
| `update_delete` | `single_cf` | `ingest_cdc_with_merge_single_family()` |
| `update_delete` | `multi_cf` | `ingest_cdc_with_merge_multi_family()` |

---

In [ ]:
import json
import os
from urllib.parse import quote

# Configuration file path (adjust as needed)
config_file = "/Users/robert.lee/github/lakeflow-community-connectors/sources/cockroachdb/.env/cockroachdb_cdc_tutorial_config_update_delete_multi_cf.json"

#config_file = "/Users/robert.lee/github/lakeflow-community-connectors/sources/cockroachdb/.env/cockroachdb_cdc_tutorial_config_update_delete_single_cf.json"

#config_file = "/Users/robert.lee/github/lakeflow-community-connectors/sources/cockroachdb/.env/cockroachdb_cdc_tutorial_config.json"

# Try to load from file, fallback to embedded config
try:
    with open(config_file, 'r') as f:
        config = json.load(f)
    print(f"✅ Configuration loaded from: {config_file}")
except Exception as e:
    print(f"ℹ️  Using embedded configuration (config file error: {e})")
    config = None

# Embedded configuration (fallback)
if config is None:
    config = {
      "cockroachdb": {
        "host": "replace_me",
        "port": 26257,
        "user": "replace_me",
        "password": "replace_me",
        "database": "defaultdb"
      },
      "cockroachdb_source": {
        "catalog": "defaultdb",
        "schema": "public",
        "table_name": "usertable",
        "_schema_note": "Default schema is 'public'. Table uses YCSB structure (ycsb_key, field0-9)",
      },
      "azure_storage": {
        "account_name": "replace_me",
        "account_key": "replace_me",
        "container_name": "changefeed-events"
      },
      "databricks_target": {
        "catalog": "main",
        "schema": "replace_me",
        "table_name": "usertable",
      },
      "cdc_config": {
        "mode": "append_only",
        "column_family_mode": "multi_cf",
        "primary_key_columns": ["ycsb_key"],
        "auto_suffix_mode_family": True,
      },
      "workload_config": {
        "snapshot_count": 10,
        "insert_count": 10,
        "update_count": 9,
        "delete_count": 8,
      }
    }


In [ ]:
from urllib.parse import quote

# Extract configuration values
cockroachdb_host = config["cockroachdb"]["host"]
cockroachdb_port = config["cockroachdb"]["port"]
cockroachdb_user = config["cockroachdb"]["user"]
cockroachdb_password = config["cockroachdb"]["password"]
cockroachdb_database = config["cockroachdb"]["database"]

source_catalog = config["cockroachdb_source"]["catalog"]
source_schema = config["cockroachdb_source"]["schema"]
source_table = config["cockroachdb_source"]["table_name"]

storage_account_name = config["azure_storage"]["account_name"]
storage_account_key = config["azure_storage"]["account_key"]
storage_account_key_encoded = quote(storage_account_key, safe='')
container_name = config["azure_storage"]["container_name"]

target_catalog = config["databricks_target"]["catalog"]
target_schema = config["databricks_target"]["schema"]
target_table = config["databricks_target"]["table_name"]

cdc_mode = config["cdc_config"]["mode"]
column_family_mode = config["cdc_config"]["column_family_mode"]
primary_key_columns = config["cdc_config"]["primary_key_columns"]

snapshot_count = config["workload_config"]["snapshot_count"]
insert_count = config["workload_config"]["insert_count"]
update_count = config["workload_config"]["update_count"]
delete_count = config["workload_config"]["delete_count"]

# Auto-suffix table names with mode and column family if enabled
auto_suffix = config["cdc_config"].get("auto_suffix_mode_family", False)
if auto_suffix:
    suffix = f"_{cdc_mode}_{column_family_mode}"
    
    # Add suffix to source_table if not already present
    if not source_table.endswith(suffix):
        source_table = f"{source_table}{suffix}"
    
    # Add suffix to target_table if not already present
    if not target_table.endswith(suffix):
        target_table = f"{target_table}{suffix}"

    # Update config dict with suffixed table names
    config["cockroachdb_source"]["table_name"] = source_table
    config["databricks_target"]["table_name"] = target_table

print("✅ Configuration loaded")
print(f"   CDC Processing Mode: {cdc_mode}")
print(f"   Column Family Mode: {column_family_mode}")
print(f"   Primary Keys: {primary_key_columns}")
print(f"   Target Table: {target_table}")
print(f"   CDC Workload: {snapshot_count} snapshot → +{insert_count} INSERTs, ~{update_count} UPDATEs, -{delete_count} DELETEs")


In [ ]:
%pip install pg8000 azure-storage-blob --quiet
print("✅ Dependencies installed")

In [ ]:
import pg8000
import ssl

def get_cockroachdb_connection():
    """Create connection to CockroachDB using pg8000"""
    # Create SSL context (required for CockroachDB Cloud)
    ssl_context = ssl.create_default_context()
    ssl_context.check_hostname = False
    ssl_context.verify_mode = ssl.CERT_NONE
    
    # Parse host (in case port is accidentally included in host string)
    host = cockroachdb_host.split(':')[0] if ':' in cockroachdb_host else cockroachdb_host
    
    conn = pg8000.connect(
        user=cockroachdb_user,
        password=cockroachdb_password,
        host=host,
        port=cockroachdb_port,
        database=cockroachdb_database,
        ssl_context=ssl_context
    )
    return conn

# Test connection
try:
    conn = get_cockroachdb_connection()
    with conn.cursor() as cur:
        cur.execute("SELECT version()")
        version = cur.fetchone()[0]
    conn.close()
    
    print("✅ Connected to CockroachDB")
    print(f"   Version: {version[:50]}...")
except Exception as e:
    print(f"❌ Connection failed: {e}")
    raise

In [ ]:
from azure.storage.blob import BlobServiceClient
from datetime import datetime
import time

def get_table_stats(conn, table_name):
    """
    Get min key, max key, and count for a table.
    
    Args:
        conn: Database connection
        table_name: Name of the table
    
    Returns:
        dict with 'min_key', 'max_key', 'count', 'is_empty'
    """
    with conn.cursor() as cur:
        cur.execute(f"SELECT MIN(ycsb_key), MAX(ycsb_key), COUNT(*) FROM {table_name}")
        result = cur.fetchone()
        min_key, max_key, count = result
        
        return {
            'min_key': min_key,
            'max_key': max_key,
            'count': count,
            'is_empty': min_key is None and max_key is None
        }


def check_azure_files(storage_account_name, storage_account_key, container_name, 
                      source_catalog, source_schema, source_table, target_table, 
                      verbose=True):
    """
    Check for changefeed files in Azure Blob Storage.
    
    Args:
        storage_account_name: Azure storage account name
        storage_account_key: Azure storage account key
        container_name: Azure container name
        source_catalog: CockroachDB catalog (database)
        source_schema: CockroachDB schema
        source_table: Source table name
        target_table: Target table name
        verbose: Print detailed output
    
    Returns:
        dict with 'data_files' and 'resolved_files' lists
    """
    # Connect to Azure
    connection_string = f"DefaultEndpointsProtocol=https;AccountName={storage_account_name};AccountKey={storage_account_key};EndpointSuffix=core.windows.net"
    blob_service = BlobServiceClient.from_connection_string(connection_string)
    container_client = blob_service.get_container_client(container_name)
    
    # Build path - list ALL files recursively under this changefeed path
    prefix = f"parquet/{source_catalog}/{source_schema}/{source_table}/{target_table}/"
    
    # List all blobs recursively (no date filtering)
    blobs = list(container_client.list_blobs(name_starts_with=prefix))
    
    # Categorize files (using same filtering logic as cockroachdb.py)
    # Data files: .parquet files, excluding:
    #   - .RESOLVED files (CDC watermarks)
    #   - _metadata/ directory (schema files)
    #   - Files starting with _ (_SUCCESS, _committed_*, etc.)
    data_files = [
        b for b in blobs 
        if b.name.endswith('.parquet') 
        and '.RESOLVED' not in b.name
        and '/_metadata/' not in b.name
        and not b.name.split('/')[-1].startswith('_')
    ]
    resolved_files = [b for b in blobs if '.RESOLVED' in b.name]
    
    if verbose:
        print(f"📁 Files in Azure changefeed path:")
        print(f"   Path: {prefix}")
        print(f"   📄 Data files: {len(data_files)}")
        print(f"   🕐 Resolved files: {len(resolved_files)}")
        print(f"   📊 Total: {len(blobs)}")
        
        if data_files:
            print(f"\n   Example data file:")
            print(f"   {data_files[0].name}")
    
    return {
        'data_files': data_files,
        'resolved_files': resolved_files,
        'total': len(blobs)
    }


def wait_for_changefeed_files(storage_account_name, storage_account_key, container_name,
                               source_catalog, source_schema, source_table, target_table,
                               max_wait=120, check_interval=5, stabilization_wait=5):
    """
    Wait for changefeed files to appear in Azure with timeout and stabilization period.
    
    This function:
    1. Polls Azure until first file(s) appear
    2. Once files are detected, waits for additional files (important for column families)
    3. Exits when no new files appear for 'stabilization_wait' seconds
    
    Args:
        max_wait: Maximum seconds to wait for initial files (default: 120)
        check_interval: Seconds between checks (default: 5)
        stabilization_wait: Seconds to wait for file count to stabilize (default: 5)
                           Important for column family mode where multiple files are written
    
    Returns:
        True if files found, False if timeout
    """
    print(f"⏳ Waiting for initial snapshot files to appear in Azure...")
    
    elapsed = 0
    files_found = False
    last_file_count = 0
    stable_elapsed = 0
    
    while elapsed < max_wait:
        result = check_azure_files(
            storage_account_name, storage_account_key, container_name,
            source_catalog, source_schema, source_table, target_table,
            verbose=False
        )
        
        current_file_count = len(result['data_files'])
        
        if not files_found and current_file_count > 0:
            # First files detected - switch to stabilization mode
            files_found = True
            last_file_count = current_file_count
            stable_elapsed = 0
            print(f"\n✅ First files appeared after {elapsed} seconds!")
            print(f"   Found {current_file_count} file(s) so far...")
            print(f"   Waiting {stabilization_wait}s for more files (column family fragments)...")
        
        elif files_found:
            # In stabilization mode - check if file count is stable
            if current_file_count > last_file_count:
                # More files arrived - reset stabilization timer
                print(f"   📄 File count increased: {last_file_count} → {current_file_count}")
                last_file_count = current_file_count
                stable_elapsed = 0
            else:
                # File count unchanged - increment stabilization timer
                stable_elapsed += check_interval
                
                if stable_elapsed >= stabilization_wait:
                    # Stabilization period complete - all files have landed
                    print(f"\n✅ File count stable at {current_file_count} for {stabilization_wait}s")
                    print(f"   Total wait time: {elapsed + stable_elapsed}s")
                    print(f"   Example: {result['data_files'][0].name}")
                    return True
        
        if not files_found:
            print(f"   Checking... ({elapsed}s elapsed)", end='\r')
        
        time.sleep(check_interval)
        elapsed += check_interval
    
    if files_found:
        # Files were found but stabilization didn't complete within max_wait
        print(f"\n⚠️  Timeout after {max_wait}s (found {last_file_count} files but more may still be generating)")
    else:
        print(f"\n⚠️  Timeout after {max_wait}s - no files appeared")
    
    print(f"   Run Cell 11 to check manually")
    return files_found  # Return True if we found at least some files



def get_column_sum(conn, table_name, column_name):
    """
    Get the sum of a numeric column in a table.
    Text columns have non-numeric characters stripped before casting.
    
    Args:
        conn: Database connection
        table_name: Name of the table
        column_name: Name of the column to sum
    
    Returns:
        Sum of the column (handles mixed text/numeric values)
    """
    with conn.cursor() as cur:
        # Strip non-numeric chars, handle empty strings, cast to BIGINT
        cur.execute(f"""
            SELECT SUM(
                CASE 
                    WHEN regexp_replace({column_name}::TEXT, '[^0-9]', '', 'g') = '' THEN 0
                    ELSE regexp_replace({column_name}::TEXT, '[^0-9]', '', 'g')::BIGINT
                END
            ) 
            FROM {table_name}
        """)
        result = cur.fetchone()
        return result[0]



def get_column_sum_spark(df, column_name):
    """
    Get the sum of a numeric column in a Spark DataFrame.
    Text columns have non-numeric characters stripped before casting.
    
    Args:
        df: Spark DataFrame
        column_name: Name of the column to sum
    
    Returns:
        Sum of the column (handles mixed text/numeric values)
    """
    from pyspark.sql import functions as F
    
    result = df.select(
        F.sum(
            F.when(
                F.regexp_replace(F.col(column_name).cast('string'), '[^0-9]', '') == '',
                0
            ).otherwise(
                F.regexp_replace(F.col(column_name).cast('string'), '[^0-9]', '').cast('bigint')
            )
        ).alias('sum')
    ).collect()[0]['sum']
    
    return result


print("✅ Helper functions loaded (CockroachDB & Azure)")

In [ ]:
def merge_column_family_fragments(
    df,
    primary_key_columns,
    spark=None,
    metadata_columns=None,
    debug=False,
    is_streaming=None,
    deduplicate_to_latest_state=False
):
    """
    Merge column family fragments into complete rows.
    
    When split_column_families=true, CockroachDB writes one Parquet file per column family,
    resulting in multiple fragment records per logical row. This function merges these
    fragments by grouping on primary key and taking the first non-null value for each column.
    
    **Streaming vs Batch Mode:**
    - **Streaming DataFrames** (from Autoloader): Always applies merge (can't detect beforehand)
    - **Batch DataFrames** (from spark.read): Auto-detects fragmentation, skips if not needed
    - Set `is_streaming=True` to force streaming mode (skips detection)
    - Set `is_streaming=False` to force batch mode (enables detection)
    
    **Deduplication Mode (NEW):**
    - `deduplicate_to_latest_state=False` (default): Merges fragments within same event, preserves all events
    - `deduplicate_to_latest_state=True`: Coalesces columns across time + deduplicates to latest state
      * Use when you have multiple UPDATE events for the same key
      * Preserves old values for columns not touched by newer events
      * Example: Event1 has field3=3, Event2 updates field0 but leaves field3=NULL
        → Result keeps field3=3 from Event1 (not NULL from Event2)
    
    Args:
        df: Spark DataFrame with potential column family fragments
        primary_key_columns: List of primary key column names (e.g., ['ycsb_key'])
        spark: SparkSession (optional, for backward compatibility)
        metadata_columns: Optional list of metadata columns to preserve
                         (default: __crdb__*, _cdc_*, _source_*, _rescued_data)
        debug: Enable debug output showing merge statistics
        is_streaming: Optional boolean to force streaming/batch mode
                     (default: auto-detect based on df.isStreaming)
        deduplicate_to_latest_state: If True, coalesce columns across time and deduplicate to latest row
                                    (default: False - preserves all CDC events)
        
    Returns:
        Merged Spark DataFrame with complete rows
        
    Example (Standard Mode):
        ```python
        df_merged = merge_column_family_fragments(
            df_raw,
            primary_key_columns=['ycsb_key'],
            debug=True
        )
        ```
        
    Example (Deduplication Mode - NEW):
        ```python
        df_latest = merge_column_family_fragments(
            df_staging,
            primary_key_columns=['ycsb_key'],
            deduplicate_to_latest_state=True,  # ← Preserves old values
            debug=True
        )
        ```
    """
    from pyspark.sql import functions as F
    from pyspark.sql.window import Window
    
    # Auto-detect streaming mode if not explicitly set
    if is_streaming is None:
        is_streaming = df.isStreaming
    
    # Default metadata columns to preserve
    if metadata_columns is None:
        metadata_columns = [
            '__crdb__event_type', '__crdb__updated', '_rescued_data',
            '_cdc_operation', '_cdc_timestamp', '_cdc_updated', '_source_file', '_processing_time',
            '_metadata',  # Unity Catalog metadata
            'after', 'before', 'key', 'updated',  # JSON envelope columns
            '_after_json', '_before_json', '_debug_after_first_10', '_debug_before_first_10'
        ]
    
    # Get all columns from DataFrame
    all_columns = df.columns
    
    # Identify data columns (everything except PK and metadata)
    data_columns = [
        col for col in all_columns 
        if col not in primary_key_columns and col not in metadata_columns
        and not col.startswith('__crdb__')
        and not col.startswith('_cdc_')
        and not col.startswith('_source_')
        and not col.startswith('_rescued_')
    ]
    
    if debug:
        mode_str = "Streaming" if is_streaming else "Batch"
        print(f"\n🔍 Column Family Merge ({mode_str} Mode)")
        print(f"   Primary key columns: {primary_key_columns}")
        print(f"   Data columns: {len(data_columns)} columns")
        if len(data_columns) <= 10:
            print(f"     {data_columns}")
        else:
            print(f"     {data_columns[:5]}... (showing first 5)")
    
    # For batch mode: Check if merge is needed
    if not is_streaming:
        try:
            # Determine timestamp column for fragmentation detection
            timestamp_col_for_check = None
            if '_cdc_timestamp' in all_columns:
                timestamp_col_for_check = '_cdc_timestamp'
            elif '_cdc_updated' in all_columns:
                timestamp_col_for_check = '_cdc_updated'
            elif '__crdb__updated' in all_columns:
                timestamp_col_for_check = '__crdb__updated'
            elif 'updated' in all_columns:
                timestamp_col_for_check = 'updated'
            
            # Try to detect fragmentation
            total_rows = df.count()
            
            if timestamp_col_for_check and '_cdc_operation' in all_columns:
                unique_events = df.select(primary_key_columns + [timestamp_col_for_check, '_cdc_operation']).distinct().count()
            elif timestamp_col_for_check:
                unique_events = df.select(primary_key_columns + [timestamp_col_for_check]).distinct().count()
            elif '_cdc_operation' in all_columns:
                unique_events = df.select(primary_key_columns + ['_cdc_operation']).distinct().count()
            else:
                unique_events = df.select(primary_key_columns).distinct().count()
            
            if debug:
                print(f"\n📊 Fragmentation Detection:")
                print(f"   Total rows: {total_rows:,}")
                print(f"   Unique events: {unique_events:,}")
                print(f"   Duplication ratio: {total_rows / unique_events if unique_events > 0 else 0:.1f}x")
            
            # If no duplicates, return original DataFrame
            if total_rows == unique_events:
                if debug:
                    print(f"\n✅ No column family fragmentation detected")
                return df
        except Exception as e:
            if debug:
                print(f"\n⚠️  Detection failed (treating as streaming): {e}")
    
    # ============================================================================
    # DEDUPLICATION MODE: Coalesce columns across time + deduplicate to latest state
    # ============================================================================
    if deduplicate_to_latest_state:
        if debug:
            print(f"\n🔄 Applying cross-time coalescing + deduplication...")
            print(f"   (Preserves latest non-NULL value per column across all events)")
        
        # Determine timestamp column for ordering
        timestamp_col_for_coalesce = None
        if '_cdc_timestamp' in all_columns:
            timestamp_col_for_coalesce = '_cdc_timestamp'
        elif '_cdc_updated' in all_columns:
            timestamp_col_for_coalesce = '_cdc_updated'
        elif '__crdb__updated' in all_columns:
            timestamp_col_for_coalesce = '__crdb__updated'
        elif 'updated' in all_columns:
            timestamp_col_for_coalesce = 'updated'
        
        if not timestamp_col_for_coalesce:
            if debug:
                print(f"   ⚠️  No timestamp column found - cannot coalesce across time")
                print(f"   Falling back to standard merge mode")
        else:
            # Step 1: Coalesce columns across time (keep latest non-NULL value per column)
            if debug:
                print(f"   Step 1: Coalescing columns by primary keys: {primary_key_columns}...")
                print(f"           Using last_value(col, ignorenulls=True) per column")
            
            # Window spec: partition by PK, order by timestamp, look at all rows
            window_spec_coalesce = (Window.partitionBy(*primary_key_columns)
                .orderBy(F.col(timestamp_col_for_coalesce))
                .rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing))
            
            # For each data column, coalesce to latest non-NULL value
            for col in data_columns:
                df = df.withColumn(
                    col,
                    F.last(F.col(col), ignorenulls=True).over(window_spec_coalesce)
                )
            
            # Also coalesce _cdc_operation to the LATEST value (for DELETE handling)
            if '_cdc_operation' in all_columns:
                df = df.withColumn(
                    "_cdc_operation",
                    F.last(F.col("_cdc_operation"), ignorenulls=True).over(window_spec_coalesce)
                )
            
            if debug:
                print(f"   ✅ Columns coalesced (latest non-NULL value per column)")
            
            # Step 2: Deduplicate by primary key (keep LATEST row, which now has ALL coalesced columns)
            if debug:
                print(f"   Step 2: Deduplicating by primary keys: {primary_key_columns}...")
            
            window_spec_dedup = Window.partitionBy(*primary_key_columns).orderBy(F.col(timestamp_col_for_coalesce).desc())
            df_merged = (df
                .withColumn("_row_num", F.row_number().over(window_spec_dedup))
                .filter(F.col("_row_num") == 1)
                .drop("_row_num")
            )
            
            if debug:
                print(f"   ✅ Deduplication complete!")
                print(f"      Result: Latest state per primary key with all column values preserved")
            
            return df_merged
    
    # ============================================================================
    # STANDARD MODE: Merge fragments within events, preserve all CDC events
    # ============================================================================
    
    # Determine which timestamp column to use for grouping
    timestamp_col = None
    if '_cdc_timestamp' in all_columns:
        timestamp_col = '_cdc_timestamp'
    elif '_cdc_updated' in all_columns:
        timestamp_col = '_cdc_updated'
    elif '__crdb__updated' in all_columns:
        timestamp_col = '__crdb__updated'
    elif 'updated' in all_columns:
        timestamp_col = 'updated'
    
    # Build aggregation expressions
    agg_exprs = []
    
    # Determine grouping columns
    if timestamp_col and '_cdc_operation' in all_columns:
        # Group by PK + timestamp + operation to preserve all distinct CDC events
        group_by_cols = primary_key_columns + [timestamp_col, '_cdc_operation']
        
        # For aggregation, use first() with ignorenulls to combine NULL values from fragments
        for col in data_columns:
            agg_exprs.append(F.first(col, ignorenulls=True).alias(col))
        
        # Metadata columns: also use first()
        for col in metadata_columns:
            if col in all_columns and col not in group_by_cols:
                agg_exprs.append(F.first(col, ignorenulls=True).alias(col))
    elif timestamp_col:
        # Fallback: Group by PK + timestamp only
        group_by_cols = primary_key_columns + [timestamp_col]
        
        for col in data_columns:
            agg_exprs.append(F.first(col, ignorenulls=True).alias(col))
        
        for col in metadata_columns:
            if col in all_columns and col not in group_by_cols:
                agg_exprs.append(F.first(col, ignorenulls=True).alias(col))
    else:
        # No timestamp column - group by PK + operation if available
        if '_cdc_operation' in all_columns:
            group_by_cols = primary_key_columns + ['_cdc_operation']
        else:
            group_by_cols = primary_key_columns
        
        for col in data_columns:
            agg_exprs.append(F.first(col, ignorenulls=True).alias(col))
        
        for col in metadata_columns:
            if col in all_columns and col not in group_by_cols:
                agg_exprs.append(F.first(col, ignorenulls=True).alias(col))
    
    # Group and aggregate
    if not agg_exprs:
        # No columns to aggregate beyond grouping key
        if debug:
            print(f"\n⚠️  No additional columns to aggregate beyond grouping key")
        df_merged = df.select(*group_by_cols).distinct()
    else:
        df_merged = df.groupBy(*group_by_cols).agg(*agg_exprs)
    
    if debug:
        print(f"\n✅ Merge transformation applied!")
    
    return df_merged


In [ ]:
# Create table structure based on column_family_mode:
# - single_cf: 1 column family (default, better performance)
# - multi_cf: 3 column families (for testing split_column_families=true)

if column_family_mode == "multi_cf":
    # Create table with MULTIPLE column families for testing split_column_families=true
    create_table_sql = f"""
    CREATE TABLE IF NOT EXISTS {source_table} (
        ycsb_key INT PRIMARY KEY,
        -- Family 1: Frequently accessed fields
        field0 TEXT,
        field1 TEXT,
        field2 TEXT,
        FAMILY frequently_read (ycsb_key, field0, field1, field2),
        
        -- Family 2: Medium-frequency fields
        field3 TEXT,
        field4 TEXT,
        field5 TEXT,
        FAMILY medium_read (field3, field4, field5),
        
        -- Family 3: Rarely accessed fields
        field6 TEXT,
        field7 TEXT,
        field8 TEXT,
        field9 TEXT,
        FAMILY rarely_read (field6, field7, field8, field9)
    )
    """
    family_info = "3 column families (frequently_read, medium_read, rarely_read)"
else:
    # Create table with SINGLE column family (default)
    create_table_sql = f"""
    CREATE TABLE IF NOT EXISTS {source_table} (
        ycsb_key INT PRIMARY KEY,
        field0 TEXT,
        field1 TEXT,
        field2 TEXT,
        field3 TEXT,
        field4 TEXT,
        field5 TEXT,
        field6 TEXT,
        field7 TEXT,
        field8 TEXT,
        field9 TEXT
    )
    """
    family_info = "1 column family (default primary)"

conn = get_cockroachdb_connection()
try:
    with conn.cursor() as cur:
        cur.execute(create_table_sql)
        conn.commit()
    print(f"✅ Table '{source_table}' created (or already exists)")
    print(f"   Column Family Mode: {column_family_mode}")
    print(f"   Column families: {family_info}")
finally:
    conn.close()

In [ ]:
conn = get_cockroachdb_connection()
try:
    # Check if table is empty using helper function
    stats = get_table_stats(conn, source_table)
    
    if stats['is_empty']:
        # Table is empty - insert snapshot data
        print(f"📊 Table is empty. Inserting {snapshot_count} initial rows (snapshot phase)...")
        
        with conn.cursor() as cur:
            # Use generate_series for efficient bulk insert
            insert_sql = f"""
            INSERT INTO {source_table} 
            (ycsb_key, field0, field1, field2, field3, field4, field5, field6, field7, field8, field9)
            SELECT 
                i AS ycsb_key,
                'snapshot_value_' || i || '_0' AS field0,
                'snapshot_value_' || i || '_1' AS field1,
                'snapshot_value_' || i || '_2' AS field2,
                'snapshot_value_' || i || '_3' AS field3,
                'snapshot_value_' || i || '_4' AS field4,
                'snapshot_value_' || i || '_5' AS field5,
                'snapshot_value_' || i || '_6' AS field6,
                'snapshot_value_' || i || '_7' AS field7,
                'snapshot_value_' || i || '_8' AS field8,
                'snapshot_value_' || i || '_9' AS field9
            FROM generate_series(0, %s - 1) AS i
            """
            
            cur.execute(insert_sql, (snapshot_count,))
            conn.commit()
        
        print(f"✅ Sample data inserted using generate_series")
        print(f"   Rows inserted: {snapshot_count} (keys 0 to {snapshot_count - 1})")
    else:
        # Table already has data - skip insert
        print(f"ℹ️  Table already contains data - skipping snapshot insert")
        print(f"   Current key range: {stats['min_key']} to {stats['max_key']}")
        print(f"   Tip: If you want to re-run the snapshot, drop the table first (see Cleanup cells)")
finally:
    conn.close()

In [ ]:
# Build Azure Blob Storage URI with table-specific path
# Note: For Azure, path goes in URI (not as path_prefix query parameter like S3)
path = f"parquet/{source_catalog}/{source_schema}/{source_table}/{target_table}"
changefeed_path = f"azure://{container_name}/{path}?AZURE_ACCOUNT_NAME={storage_account_name}&AZURE_ACCOUNT_KEY={storage_account_key_encoded}"

# Build changefeed options based on column_family_mode
if column_family_mode == "multi_cf":
    # Include split_column_families for multi-family mode
    changefeed_options = """
    format='parquet',
    updated,
    resolved='10s',
    split_column_families
"""
else:
    # Standard options for single-family mode
    changefeed_options = """
    format='parquet',
    updated,
    resolved='10s'
"""

# Create changefeed SQL
create_changefeed_sql = f"""
CREATE CHANGEFEED FOR TABLE {source_table}
INTO '{changefeed_path}'
WITH {changefeed_options}
"""

conn = get_cockroachdb_connection()
try:
    with conn.cursor() as cur:
        # Check for existing changefeed with THIS specific destination path
        # (checks for source table AND full path to ensure uniqueness)
        path_pattern = f"%{source_table}%{source_catalog}/{source_schema}/{source_table}/{target_table}%"
        
        cur.execute("""
            SELECT job_id, status, description
            FROM [SHOW CHANGEFEED JOBS] 
            WHERE description LIKE %s
            AND status IN ('running', 'paused')
            LIMIT 1
        """, (path_pattern,))
        
        existing = cur.fetchone()
        
        if existing:
            job_id, status, description = existing
            print(f"✅ Changefeed already exists for this source → target mapping")
            print(f"   Job ID: {job_id}")
            print(f"   Status: {status}")
            print(f"   Source: {source_catalog}.{source_schema}.{source_table}")
            print(f"   Target path: .../{source_table}/{target_table}/")
            if column_family_mode == "multi_cf":
                print(f"   Expected: Column family fragments")
            print(f"")
            print(f"💡 Tip: Run Cell 10 to generate UPDATE/DELETE events")
            print(f"   Then check Cell 11 to verify new files appear")
        else:
            # Create new changefeed
            cur.execute(create_changefeed_sql)
            result = cur.fetchone()
            job_id = result[0]
            
            print(f"✅ Changefeed created")
            print(f"   Job ID: {job_id}")
            print(f"   Source: {source_catalog}.{source_schema}.{source_table}")
            print(f"   Target path: .../{source_table}/{target_table}/")
            print(f"   Format: Parquet")
            if column_family_mode == "multi_cf":
                print(f"   Split column families: TRUE (fragments will be generated)")
            else:
                print(f"   Split column families: FALSE (single file per event)")
            print(f"   Destination: Azure Blob Storage")
            print(f"")
            
            # Wait for files to appear using helper function
            wait_for_changefeed_files(
                storage_account_name, storage_account_key, container_name,
                source_catalog, source_schema, source_table, target_table,
                max_wait=300, check_interval=5
            )
finally:
    conn.close()

In [ ]:
import time
from datetime import datetime

# Capture baseline file count BEFORE generating CDC events
print("📊 Capturing baseline file count...")
result_before = check_azure_files(
    storage_account_name, storage_account_key, container_name,
    source_catalog, source_schema, source_table, target_table,
    verbose=False
)
files_before = len(result_before['data_files'])
print(f"   Current files: {files_before}")
print()

conn = get_cockroachdb_connection()
try:
    # Get current table state using helper function
    stats_before = get_table_stats(conn, source_table)
    min_key = stats_before['min_key']
    max_key = stats_before['max_key']
    count_before = stats_before['count']
    
    print(f"📊 Current table state:")
    print(f"   Min key: {min_key}, Max key: {max_key}, Total rows: {count_before}")
    print()
    
    with conn.cursor() as cur:
        
        # 1. INSERT: Add new rows starting from max_key + 1 (using generate_series)
        print(f"➕ Running {insert_count} INSERTs (keys {max_key + 1} to {max_key + insert_count})...")
        insert_sql = f"""
        INSERT INTO {source_table} 
        (ycsb_key, field0, field1, field2, field3, field4, field5, field6, field7, field8, field9)
        SELECT 
            i AS ycsb_key,
            'inserted_value_' || i || '_0' AS field0,
            'inserted_value_' || i || '_1' AS field1,
            'inserted_value_' || i || '_2' AS field2,
            'inserted_value_' || i || '_3' AS field3,
            'inserted_value_' || i || '_4' AS field4,
            'inserted_value_' || i || '_5' AS field5,
            'inserted_value_' || i || '_6' AS field6,
            'inserted_value_' || i || '_7' AS field7,
            'inserted_value_' || i || '_8' AS field8,
            'inserted_value_' || i || '_9' AS field9
        FROM generate_series(%s, %s) AS i
        """
        cur.execute(insert_sql, (max_key + 1, max_key + insert_count))
        
        # 2. UPDATE: Update existing rows starting from min_key (single UPDATE statement)
        print(f"📝 Running {update_count} UPDATEs (keys {min_key} to {min_key + update_count - 1})...")
        timestamp = int(time.time())
        cur.execute(f"""
            UPDATE {source_table}
            SET field0 = %s
            WHERE ycsb_key >= %s AND ycsb_key < %s
        """, (f"updated_at_{timestamp}", min_key, min_key + update_count))
        
        # 3. DELETE: Delete oldest rows starting from min_key (single DELETE)
        delete_max = min_key + delete_count - 1
        print(f"🗑️  Running {delete_count} DELETEs (keys {min_key} to {delete_max})...")
        cur.execute(f"""
            DELETE FROM {source_table}
            WHERE ycsb_key >= %s AND ycsb_key <= %s
        """, (min_key, delete_max))
        
        conn.commit()
    
    # Get final table state using helper function
    stats_after = get_table_stats(conn, source_table)
    min_key_after = stats_after['min_key']
    max_key_after = stats_after['max_key']
    count_after = stats_after['count']
    
    print(f"\n✅ Workload complete")
    print(f"   Inserts: {insert_count}")
    print(f"   Updates: {update_count}")
    print(f"   Deletes: {delete_count}")
    print(f"   Before: {count_before} rows (keys {min_key}-{max_key})")
    print(f"   After:  {count_after} rows (keys {min_key_after}-{max_key_after})")
    print(f"   Net change: {count_after - count_before:+d} rows")
    print(f"")
    
    # Wait for new CDC files to appear in Azure (positive confirmation)
    print(f"⏳ Waiting for new CDC files to appear in Azure...")
    print(f"   Baseline: {files_before} files")
    print()
    
    # Poll for new files (max 90 seconds)
    max_wait = 90
    check_interval = 10
    elapsed = 0
    
    while elapsed < max_wait:
        result = check_azure_files(
            storage_account_name, storage_account_key, container_name,
            source_catalog, source_schema, source_table, target_table,
            verbose=False
        )
        files_now = len(result['data_files'])
        
        if files_now > files_before:
            print(f"✅ New CDC files appeared after {elapsed} seconds!")
            print(f"   Baseline (before workload): {files_before} files")
            print(f"   Current (after workload): {files_now} files")
            print(f"   New files generated: {files_now - files_before}")
            break
        
        print(f"   Checking... ({elapsed}s elapsed, baseline: {files_before} files)", end='\r')
        time.sleep(check_interval)
        elapsed += check_interval
    else:
        print(f"\n⚠️  Timeout after {max_wait}s - files may still be flushing")
        print(f"   Run Cell 11 to check manually")
finally:
    conn.close()

In [ ]:
# Use the helper function from Cell 4 to check for files
result = check_azure_files(
    storage_account_name, storage_account_key, container_name,
    source_catalog, source_schema, source_table, target_table,
    verbose=True
)

# Provide guidance
if len(result['data_files']) == 0:
    print(f"\n⚠️  No data files found yet.")
    print(f"   💡 Possible reasons:")
    print(f"   - Changefeed not created yet (run Cell 9)")
    print(f"   - Path configuration mismatch (check Cell 1 variables)")
    print(f"   - Azure credentials issue (check External Location)")
else:
    print(f"\n✅ Files are ready! Proceed to Cell 10 to read with Databricks.")

In [ ]:
# Select and run CDC ingestion function based on both modes (from Cell 1)
# Functions are defined in Cell 5

print(f"🔷 CDC Configuration:")
print(f"   Processing Mode: {cdc_mode}")
print(f"   Column Family Mode: {column_family_mode}")
print()

# Select function based on BOTH cdc_mode and column_family_mode
if cdc_mode == "append_only" and column_family_mode == "single_cf":
    print(f"📘 Running: ingest_cdc_append_only_single_family()")
    print(f"   - All CDC events will be stored as rows")
    print(f"   - No column family merging needed\n")
    
    query = ingest_cdc_append_only_single_family(
        storage_account_name=storage_account_name,
        container_name=container_name,
        source_catalog=source_catalog,
        source_schema=source_schema,
        source_table=source_table,
        target_catalog=target_catalog,
        target_schema=target_schema,
        target_table=target_table,
        spark=spark
    )

elif cdc_mode == "append_only" and column_family_mode == "multi_cf":
    print(f"📙 Running: ingest_cdc_append_only_multi_family()")
    print(f"   - All CDC events will be stored as rows")
    print(f"   - Column family fragments will be merged\n")
    
    if not primary_key_columns:
        raise ValueError("primary_key_columns required for multi_cf mode")
    
    query = ingest_cdc_append_only_multi_family(
        storage_account_name=storage_account_name,
        container_name=container_name,
        source_catalog=source_catalog,
        source_schema=source_schema,
        source_table=source_table,
        target_catalog=target_catalog,
        target_schema=target_schema,
        target_table=target_table,
        primary_key_columns=primary_key_columns,
        spark=spark
    )

elif cdc_mode == "update_delete" and column_family_mode == "single_cf":
    print(f"📗 Running: ingest_cdc_with_merge_single_family()")
    print(f"   - MERGE logic applied (UPDATE/DELETE processed)")
    print(f"   - No column family merging needed\n")
    
    if not primary_key_columns:
        raise ValueError("primary_key_columns required for update_delete mode")
    
    result = ingest_cdc_with_merge_single_family(
        storage_account_name=storage_account_name,
        container_name=container_name,
        source_catalog=source_catalog,
        source_schema=source_schema,
        source_table=source_table,
        target_catalog=target_catalog,
        target_schema=target_schema,
        target_table=target_table,
        primary_key_columns=primary_key_columns,
        spark=spark
    )
    
    query = result["query"]

elif cdc_mode == "update_delete" and column_family_mode == "multi_cf":
    print(f"📕 Running: ingest_cdc_with_merge_multi_family()")
    print(f"   - MERGE logic applied (UPDATE/DELETE processed)")
    print(f"   - Column family fragments will be merged\n")
    
    if not primary_key_columns:
        raise ValueError("primary_key_columns required for update_delete + multi_cf mode")
    
    result = ingest_cdc_with_merge_multi_family(
        storage_account_name=storage_account_name,
        container_name=container_name,
        source_catalog=source_catalog,
        source_schema=source_schema,
        source_table=source_table,
        target_catalog=target_catalog,
        target_schema=target_schema,
        target_table=target_table,
        primary_key_columns=primary_key_columns,
        spark=spark
    )
    
    query = result["query"]

else:
    raise ValueError(
        f"Invalid mode combination:\n"
        f"  cdc_mode='{cdc_mode}' (valid: 'append_only', 'update_delete')\n"
        f"  column_family_mode='{column_family_mode}' (valid: 'single_cf', 'multi_cf')\n"
        f"Change modes in Cell 1."
    )

# Wait for completion (if not already complete)
if cdc_mode == "append_only":
    query.awaitTermination()
    print("\n" + "=" * 80)
    print(f"✅ CDC INGESTION COMPLETE")
    print("=" * 80)
    print(f"   Mode: {cdc_mode} + {column_family_mode}")
    print(f"   Target: {target_catalog}.{target_schema}.{target_table}")
    print()
    print(f"📊 Query your data: SELECT * FROM {target_catalog}.{target_schema}.{target_table}")
else:
    # update_delete mode already completed inside the function
    print(f"📊 Query your data: SELECT * FROM {target_catalog}.{target_schema}.{target_table}")

In [ ]:
target_table_fqn = f"{target_catalog}.{target_schema}.{target_table}"

# Get total count
df = spark.read.table(target_table_fqn)
total_count = df.count()

print("📊 CDC Event Summary")
print("=" * 80)
print(f"Total rows: {total_count}")
print(f"CDC Processing Mode: {cdc_mode}")
print(f"Column Family Mode: {column_family_mode}")
print()

# Show operation breakdown (works for both modes now!)
print("Rows by last CDC operation:")
ops_df = df.groupBy("_cdc_operation").count().orderBy("_cdc_operation")
ops_df.show()

print("\n📋 Sample rows (showing first 5):")
df.select(
    "ycsb_key", 
    "field0", 
    "_cdc_operation", 
    "_cdc_timestamp"
).orderBy("_cdc_timestamp").show(5, truncate=False)

if cdc_mode == "append_only":
    print("\n✅ CDC data successfully loaded (append_only mode)")
    print("   📊 All CDC events stored as rows")
    print("   📊 _cdc_operation shows: DELETE, UPSERT for each event")
    print("   📊 Row count = all events (including DELETEs and multiple UPDATEs)")
elif cdc_mode == "update_delete":
    print("\n✅ CDC data successfully loaded (update_delete mode)")
    print("   📊 MERGE operations applied: DELETEs removed, UPDATEs applied, INSERTs added")
    print("   📊 _cdc_operation shows: UPSERT (last operation on each row)")
    print("   📊 Row count = current state (deduplicated)")

print("\n💡 Key Takeaway:")
print("   - Using pathGlobFilter to exclude .RESOLVED files avoids DECIMAL errors")
print("   - _cdc_operation is preserved in both modes for monitoring")
print("\n📍 Next: Run Cell 14 to verify source and target tables are in sync")

In [ ]:
print("🔍 Verifying source and target tables are in sync...")
print("=" * 80)

# Get source table stats (CockroachDB)
conn = get_cockroachdb_connection()
try:
    source_stats = get_table_stats(conn, source_table)
    source_sum = get_column_sum(conn, source_table, 'ycsb_key')
    
    # Get target table stats (Databricks Delta)
    target_table_fqn = f"{target_catalog}.{target_schema}.{target_table}"
    target_df = spark.read.table(target_table_fqn)
    
    # Calculate target stats using Spark
    from pyspark.sql import functions as F
    target_stats_df = target_df.agg(
        F.min("ycsb_key").alias("min_key"),
        F.max("ycsb_key").alias("max_key"),
        F.count("*").alias("count")
    ).collect()[0]
    
    target_stats = {
        'min_key': target_stats_df['min_key'],
        'max_key': target_stats_df['max_key'],
        'count': target_stats_df['count']
    }
    
    # Calculate sum using Spark helper function
    target_sum = get_column_sum_spark(target_df, 'ycsb_key')
    
    # Display comparison
    print(f"\n📊 Source Table (CockroachDB): {source_catalog}.{source_schema}.{source_table}")
    print(f"   Min key: {source_stats['min_key']}")
    print(f"   Max key: {source_stats['max_key']}")
    print(f"   Count:   {source_stats['count']}")
    print(f"   Sum (ycsb_key): {source_sum}")
    
    print(f"\n📊 Target Table (Databricks Delta): {target_table_fqn}")
    print(f"   Min key: {target_stats['min_key']}")
    print(f"   Max key: {target_stats['max_key']}")
    print(f"   Count:   {target_stats['count']}")
    print(f"   Sum (ycsb_key): {target_sum}")
    
    # Verify all numeric columns (YCSB schema: field0-9)
    print("\n📊 Column Sums Comparison (All Fields):")
    print("-" * 80)
    
    columns_to_verify = ['ycsb_key', 'field0', 'field1', 'field2', 'field3',
                         'field4', 'field5', 'field6', 'field7', 'field8', 'field9']
    
    all_columns_match = True
    for col in columns_to_verify:
        try:
            source_col_sum = get_column_sum(conn, source_table, col)
            target_col_sum = get_column_sum_spark(target_df, col)
            col_matches = source_col_sum == target_col_sum
            match_icon = "✅" if col_matches else "❌"
            
            # Format with commas for readability
            source_str = f"{source_col_sum:,}" if source_col_sum else "NULL"
            target_str = f"{target_col_sum:,}" if target_col_sum else "NULL"
            
            print(f"{match_icon} {col:12s}: Source={source_str:>20s} | Target={target_str:>20s}")
            
            if not col_matches:
                all_columns_match = False
                diff = (target_col_sum or 0) - (source_col_sum or 0)
                print(f"   ⚠️  Difference: {diff:+,}")
        except Exception as e:
            print(f"⚠️  {col:12s}: Error calculating sum - {e}")
            all_columns_match = False
    
    if all_columns_match:
        print("\n✅ All column sums match!")
    else:
        print("\n⚠️  Some column sums do not match - check data integrity")
    
finally:
    # Always close the connection, even if there's an error
    conn.close()

# Check CDC sync status (mode-aware verification)
print(f"\n{'=' * 80}")
print(f"Mode: {cdc_mode.upper()}")
print("=" * 80)

# Compare keys and counts
min_key_matches = source_stats['min_key'] == target_stats['min_key']
max_key_matches = source_stats['max_key'] == target_stats['max_key']
count_matches = source_stats['count'] == target_stats['count']
sum_matches = source_sum == target_sum

if cdc_mode == "append_only":
    # In append_only mode, max_key is the key indicator of sync
    # Min key and count are expected to differ due to deletes/updates not being applied
    
    if max_key_matches:
        print("✅ CDC PIPELINE IS WORKING!")
        print(f"   Max key matches: {source_stats['max_key']}")
        print(f"   Sum matches: {source_sum}")
        print()
        
        if not min_key_matches or not count_matches:
            print("📋 Append-Only Mode (Expected Differences):")
            if not min_key_matches:
                print(f"   ℹ️  Min key differs (Source={source_stats['min_key']}, Target={target_stats['min_key']})")
                print(f"      → This is EXPECTED: DELETE events are captured but not applied")
            if not count_matches:
                print(f"   ℹ️  Row count differs (Source={source_stats['count']}, Target={target_stats['count']})")
                print(f"      → This is EXPECTED: All CDC events (INSERT/UPDATE/DELETE) are stored as rows")
            print()
            print("💡 To apply deletes and updates:")
            print("   - Change cdc_mode='update_delete' in Cell 1")
            print("   - Or manually deduplicate using SQL window functions")
    else:
        print("⚠️  MAX KEY MISMATCH - CDC may be lagging")
        print("\n   Key Statistics:")
        print(f"   - Source max: {source_stats['max_key']}")
        print(f"   - Target max: {target_stats['max_key']}")
        print(f"   - Difference: {source_stats['max_key'] - target_stats['max_key']:+d}")
        
        print("\n   💡 Possible reasons:")
        print("   - Auto Loader hasn't picked up all files yet (re-run Cell 12)")
        print("   - MERGE logic issue (check Cell 12 output)")
        print("   - Run Example 4 in Debug Section (Cell 30) for full diagnosis")

elif cdc_mode == "update_delete":
    # In update_delete mode, ALL stats should match exactly
    # DELETE operations are applied, UPDATE operations modify existing rows
    
    if min_key_matches and max_key_matches and count_matches and sum_matches and all_columns_match:
        print("✅ CDC PIPELINE IS WORKING PERFECTLY! All statistics and column sums match.")
        print("   All statistics match:")
        print(f"   ✅ Min key: {source_stats['min_key']}")
        print(f"   ✅ Max key: {source_stats['max_key']}")
        print(f"   ✅ Count:   {source_stats['count']}")
        print(f"   ✅ Sum:     {source_sum}")
        print()
        print("📋 Update-Delete Mode:")
        print("   ✅ DELETE events are applied (rows removed)")
        print("   ✅ UPDATE events are applied (rows modified)")
        print("   ✅ INSERT events are applied (rows added)")
    else:
        print("⚠️  SYNC MISMATCH - Tables are out of sync")
        print("\n   Key Statistics:")
        if not min_key_matches:
            print(f"   ❌ Min key: Source={source_stats['min_key']}, Target={target_stats['min_key']}")
        else:
            print(f"   ✅ Min key: {source_stats['min_key']}")
        
        if not max_key_matches:
            print(f"   ❌ Max key: Source={source_stats['max_key']}, Target={target_stats['max_key']}")
        else:
            print(f"   ✅ Max key: {source_stats['max_key']}")
        
        if not count_matches:
            print(f"   ❌ Count: Source={source_stats['count']}, Target={target_stats['count']}")
            print(f"      Difference: {target_stats['count'] - source_stats['count']:+d} rows")
        else:
            print(f"   ✅ Count: {source_stats['count']}")
        
        if not sum_matches:
            print(f"   ❌ Sum: Source={source_sum}, Target={target_sum}")
            print(f"      Difference: {target_sum - source_sum:+d}")
        else:
            print(f"   ✅ Sum: {source_sum}")
        
        
        if not all_columns_match:
            print(f"   ❌ Column sums: Some column sums do not match (see above)")
        else:
            print(f"   ✅ Column sums: All match")
        print("\n   💡 Possible reasons:")
        print("   - Auto Loader hasn't picked up all files yet (re-run Cell 12)")
        print("   - MERGE logic issue (check Cell 12 output for errors)")
        print("   - DELETE rows stored as data (run Cell 16 to fix)")
        print("   - Run diagnostic cell (Cell 15) to inspect target table")

else:
    print(f"⚠️  Unknown mode: {cdc_mode}")
    print("   Cannot verify sync status")

## Optional: Cleanup

Run the cells below if you want to clean up the test resources.

In [ ]:
# ⚠️  SAFETY STOP: Cleanup Section
# This cell prevents accidental cleanup when running "Run All"
# 
# To cleanup resources, manually run each cell below INDIVIDUALLY:
#   - Cell 16: Cancel changefeed
#   - Cell 17: Drop CockroachDB source table  
#   - Cell 18: Drop Databricks target table & checkpoint
#   - Cell 19: Clear Azure changefeed data (optional - use for complete reset)

raise RuntimeError(
    "\n"
    "⚠️  CLEANUP SAFETY STOP\n"
    "\n"
    "The cells below will DELETE your resources.\n"
    "Do NOT run all cells - run each cleanup cell individually.\n"
    "\n"
    "💡 TIP: If Cell 13 shows sync issues due to old data,\n"
    "   run Cell 19 to clear Azure changefeed data completely.\n"
)

In [18]:
# CLEANUP CELL 1: CANCEL CHANGEFEED
conn = get_cockroachdb_connection()
try:
    with conn.cursor() as cur:
        # Find changefeed job for THIS specific source → target mapping
        # (matches the same path pattern used in Cell 7)
        path_pattern = f"%{source_table}%{source_catalog}/{source_schema}/{source_table}/{target_table}%"
        
        cur.execute("""
            SELECT job_id 
            FROM [SHOW CHANGEFEED JOBS] 
            WHERE description LIKE %s
            AND status IN ('running', 'paused')
            LIMIT 1
        """, (path_pattern,))
        
        result = cur.fetchone()
        if result:
            job_id = result[0]
            cur.execute(f"CANCEL JOB {job_id}")
            print(f"✅ Changefeed {job_id} cancelled")
            print(f"   Source: {source_catalog}.{source_schema}.{source_table}")
            print(f"   Target path: .../{source_table}/{target_table}/")
        else:
            print("ℹ️  No active changefeed found for this source → target mapping")
finally:
    conn.close()

✅ Changefeed 1145428509383163905 cancelled
   Source: defaultdb.public.usertable_update_delete_multi_cf
   Target path: .../usertable_update_delete_multi_cf/usertable_update_delete_multi_cf/


In [19]:
# CLEANUP CELL 2: DROP SOURCE TABLE (CockroachDB)
conn = get_cockroachdb_connection()
try:
    with conn.cursor() as cur:
        cur.execute(f"DROP TABLE IF EXISTS {source_table} CASCADE")
        conn.commit()
    print(f"✅ Table '{source_table}' dropped from CockroachDB")
finally:
    conn.close()

✅ Table 'usertable_update_delete_multi_cf' dropped from CockroachDB


In [20]:
# CLEANUP CELL 19: CLEAR AZURE CHANGEFEED DATA (Optional)
# ⚠️  WARNING: This will DELETE all changefeed data in Azure for this table!
#
# Use this when:
# - You want to start completely fresh
# - Old data from previous runs is causing sync issues
# - You changed the table schema (e.g., VARCHAR → INT)
#
# Uses Azure SDK (same as Cell 11 for checking files)

from azure.storage.blob import BlobServiceClient

# Build Azure path (must match Cell 7 changefeed path)
changefeed_path = f"parquet/{source_catalog}/{source_schema}/{source_table}/{target_table}/"

print(f"🗑️  Deleting Azure changefeed data...")
print(f"=" * 80)
print(f"Container: {container_name}")
print(f"Path: {changefeed_path}")
print()

# Connect to Azure (same as Cell 9)
connection_string = f"DefaultEndpointsProtocol=https;AccountName={storage_account_name};AccountKey={storage_account_key};EndpointSuffix=core.windows.net"
blob_service = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service.get_container_client(container_name)

# List all blobs with this prefix
print(f"🔍 Scanning for files...")
blobs = list(container_client.list_blobs(name_starts_with=changefeed_path))

if not blobs:
    print(f"ℹ️  No files found at: {changefeed_path}")
    print(f"   Files may have already been deleted, or path is incorrect")
    print()
    print(f"💡 To check what's in the container, run Cell 9")
else:
    print(f"✅ Found {len(blobs)} items to delete")
    
    # Show sample items
    data_files = [b for b in blobs if b.size > 0 and '.parquet' in b.name]
    resolved_files = [b for b in blobs if '.RESOLVED' in b.name]
    directories = [b for b in blobs if b.size == 0]
    
    print(f"   📄 Data files: {len(data_files)}")
    print(f"   🕐 Resolved files: {len(resolved_files)}")
    print(f"   📁 Directories: {len(directories)}")
    print()
    
    # Delete all blobs with this prefix
    # Note: Azure SDK doesn't have recursive delete - we list all blobs and delete each one
    print(f"🔄 Deleting {len(blobs)} items...")
    deleted = 0
    failed = 0
    
    for blob in blobs:
        try:
            container_client.delete_blob(blob.name)
            deleted += 1
            if deleted % 50 == 0:
                print(f"   Deleted {deleted}/{len(blobs)} items...", end='\r')
        except Exception as e:
            # Some errors are expected (e.g., directories already removed)
            error_str = str(e)
            if "DirectoryIsNotEmpty" not in error_str and "BlobNotFound" not in error_str:
                failed += 1
                print(f"\n   ⚠️  Failed: {blob.name[:60]}... - {e}")
    
    print(f"✅ Deleted {deleted} items from Azure                    ")
    if failed > 0:
        print(f"   ⚠️  Failed to delete {failed} items")
    
    print()
    print(f"=" * 80)
    print(f"✅ Cleanup complete!")
    print()
    print(f"💡 Next steps:")
    print(f"   1. Drop the Databricks target table (Cell 17)")
    print(f"   2. Re-run from Cell 6 (Snapshot) to start fresh")

🗑️  Deleting Azure changefeed data...
Container: changefeed-events
Path: parquet/defaultdb/public/usertable_update_delete_multi_cf/usertable_update_delete_multi_cf/

🔍 Scanning for files...
✅ Found 761 items to delete
   📄 Data files: 51
   🕐 Resolved files: 708
   📁 Directories: 2

🔄 Deleting 761 items...
✅ Deleted 759 items from Azure                    

✅ Cleanup complete!

💡 Next steps:
   1. Drop the Databricks target table (Cell 17)
   2. Re-run from Cell 6 (Snapshot) to start fresh


In [21]:
# CLEANUP CELL 3: DROP TARGET TABLE & CHECKPOINT (Databricks)
target_table_fqn = f"{target_catalog}.{target_schema}.{target_table}"
checkpoint_path = f"/checkpoints/{target_schema}_{target_table}"  # Must match Cell 10

# Drop Delta table
spark.sql(f"DROP TABLE IF EXISTS {target_table_fqn}")
print(f"✅ Delta table '{target_table_fqn}' dropped")

# Remove checkpoint
try:
    dbutils.fs.rm(checkpoint_path, True)
    print(f"✅ Checkpoint '{checkpoint_path}' removed")
except:
    print(f"ℹ️  Checkpoint not found (may have been already removed)")

print("\n✅ Cleanup complete!")

✅ Delta table 'robert_lee.robert_lee_cockroachdb.usertable_update_delete_multi_cf' dropped
✅ Checkpoint '/checkpoints/robert_lee_cockroachdb_usertable_update_delete_multi_cf' removed

✅ Cleanup complete!


In [22]:
# CLEANUP CELL 4: Complete cleanup for fresh start

# 1. Drop staging table
staging_table_fqn = f"{target_catalog}.{target_schema}.{target_table}_staging_cf"
print(f"🗑️  Dropping staging table: {staging_table_fqn}")
spark.sql(f"DROP TABLE IF EXISTS {staging_table_fqn}")

# 2. Drop target table (if not already done)
target_table_fqn = f"{target_catalog}.{target_schema}.{target_table}"
print(f"🗑️  Dropping target table: {target_table_fqn}")
spark.sql(f"DROP TABLE IF EXISTS {target_table_fqn}")

# 3. Clear checkpoint location
checkpoint_path = f"/checkpoints/{target_schema}_{target_table}_merge_cf"
print(f"🗑️  Clearing checkpoint: {checkpoint_path}")
try:
    dbutils.fs.rm(checkpoint_path, recurse=True)
    print(f"   ✅ Checkpoint cleared")
except Exception as e:
    print(f"   ℹ️  Checkpoint may not exist: {e}")

# 4. Verify cleanup
print(f"\n✅ Cleanup complete! Ready for fresh start.")
print(f"   Next: Re-run Cell 12 (ingestion)")

🗑️  Dropping staging table: robert_lee.robert_lee_cockroachdb.usertable_update_delete_multi_cf_staging_cf
🗑️  Dropping target table: robert_lee.robert_lee_cockroachdb.usertable_update_delete_multi_cf
🗑️  Clearing checkpoint: /checkpoints/robert_lee_cockroachdb_usertable_update_delete_multi_cf_merge_cf
   ✅ Checkpoint cleared

✅ Cleanup complete! Ready for fresh start.
   Next: Re-run Cell 12 (ingestion)


In [23]:
# Recreate the schema
print(f"📁 Creating schema: {target_catalog}.{target_schema}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {target_catalog}.{target_schema}")
print(f"✅ Schema created")

# Verify schema exists
schemas = spark.sql(f"SHOW SCHEMAS IN {target_catalog}").collect()
schema_names = [row['databaseName'] for row in schemas]
if target_schema in schema_names:
    print(f"✅ Verified: Schema {target_schema} exists")
else:
    print(f"❌ Schema {target_schema} not found. Available schemas: {schema_names}")

📁 Creating schema: robert_lee.robert_lee_cockroachdb
✅ Schema created
✅ Verified: Schema robert_lee_cockroachdb exists


# Debug Codes

In [24]:
# Debug helper functions are available in cockroachdb_debug.py
# Import them in Cell 15, then use the examples below:

print("💡 Debug functions available:")
print("   • quick_check_missing_keys() - Quick check for specific keys")
print("   • inspect_target_table() - Comprehensive target table analysis")
print("   • detailed_missing_keys_investigation() - Detailed key investigation")
print("\nSee the following cells for usage examples.")

💡 Debug functions available:
   • quick_check_missing_keys() - Quick check for specific keys
   • inspect_target_table() - Comprehensive target table analysis
   • detailed_missing_keys_investigation() - Detailed key investigation

See the following cells for usage examples.


In [25]:
# Example 1: Quick check for specific missing keys
from cockroachdb_debug import quick_check_missing_keys

conn = get_cockroachdb_connection()
try:
    quick_check_missing_keys(
        conn=conn,
        spark=spark,
        source_table=source_table,
        target_catalog=target_catalog,
        target_schema=target_schema,
        target_table=target_table,
        missing_keys=[17, 18, 19]  # ← Update based on Cell 14 output
    )
finally:
    conn.close()

🔍 Quick Debug: Checking missing keys...
Keys to check: [17, 18, 19]

📊 CockroachDB (usertable_update_delete_multi_cf):


ProgrammingError: {'S': 'ERROR', 'V': 'ERROR', 'C': '42P01', 'F': 'errors.go', 'L': '233', 'R': 'NewUndefinedRelationError', 'M': 'relation "usertable_update_delete_multi_cf" does not exist'}

In [ ]:
# Example 2: Comprehensive target table analysis
from cockroachdb_debug import inspect_target_table

inspect_target_table(
    spark=spark,
    target_catalog=target_catalog,
    target_schema=target_schema,
    target_table=target_table
)

In [ ]:
# Example 3: Detailed investigation of specific missing keys
from cockroachdb_debug import detailed_missing_keys_investigation

conn = get_cockroachdb_connection()
try:
    detailed_missing_keys_investigation(
        conn=conn,
        spark=spark,
        source_table=source_table,
        target_catalog=target_catalog,
        target_schema=target_schema,
        target_table=target_table,
        missing_keys=[17, 18, 19]  # ← Update based on inspect_target_table output
    )
finally:
    conn.close()

In [ ]:
# Example 4: Full diagnosis using config (Recommended for Cell 14 sync issues)
import importlib,cockroachdb_debug
importlib.reload(cockroachdb_debug)
from cockroachdb_debug import run_full_diagnosis_from_config

# Define mismatched columns from Cell 14 output
mismatched_columns = ['field3', 'field4', 'field5', 'field6', 'field7', 'field8', 'field9']

# Run full diagnosis using config from Cell 3
# This will:
#   - Refresh target DataFrame
#   - Establish CockroachDB connection
#   - Run comprehensive diagnosis
#   - Check staging table, Azure files, and row-by-row comparison
#   - Provide detailed troubleshooting recommendations
run_full_diagnosis_from_config(
    spark=spark,
    config=config,
    mismatched_columns=mismatched_columns  # Set to None if no mismatches
)